In [7]:
import os
import requests
import json
import pandas as pd
import random
import time
import re
import sqlite3
import datetime
import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util

from dotenv import load_dotenv, find_dotenv
import warnings
 
# 忽略所有的 Warning 警告
warnings.filterwarnings('ignore')
 
# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())
coze_api_key=os.environ["COZE_API_KEY"]
COZE_BOT_ID = os.environ["COZE_BOT_ID"]
COZE_BOT_ID_ROLE = os.environ["COZE_BOT_ID_RULE"]
# 加载预训练的 Sentence-BERT 模型
sbert_model = SentenceTransformer("./model")

No sentence-transformers model found with name ./model. Creating a new one with mean pooling.


In [2]:
def call_coze_api(query,con_id='123',user_id='zmx'):
    # 填充调用Coze API获得回复的代码，返回为字典
    response_data = send_request(coze_api_key, con_id, COZE_BOT_ID, user_id, query)
    response = parse_message_object(response_data)
    if response["plugin"] != "":
        ref_info = plugin_text_process(response["plugin"])
        response.update(ref_info)
    return response

def send_request(personal_access_token, con_id, bot_id, user_id, query):
    # 填充调用Coze API的具体代码，获得coze的回复，返回为json格式
    url = 'https://api.coze.cn/open_api/v2/chat'
    
    headers = {
        'Authorization': f'Bearer {personal_access_token}',
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'Host': 'api.coze.cn',
        'Connection': 'keep-alive'
    }
    
    data = {
        'conversation_id': con_id,
        'bot_id': bot_id,
        'user': user_id,
        'query': query,
        'stream': False
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    return response.json()

def parse_message_object(message_dict):
    # 解析coze API返回的结果，以字典的形式返回
    # 初始化变量以存储结果
    plugin_data = ""
    last_answer_content = ""

    # 获取messages列表
    messages = message_dict.get('messages', [])

    # 遍历messages列表
    for message in messages:
        # 提取最后一个answer的content值
        if message.get('type') == 'answer':
            last_answer_content = message.get('content', "")

        # 提取plugin数据
        if message.get('type') == 'verbose':
            content_str = message.get('content', "")
            try:
                # 尝试将字符串解析为JSON对象
                content_data = json.loads(content_str)
                # 检查msg_type是否为stream_plugin_finish
                if content_data.get('msg_type') == 'stream_plugin_finish':
                    plugin_data = json.dumps(content_data.get('data'), ensure_ascii=False)
            except (json.JSONDecodeError, TypeError):
                # 如果解析失败或data不是字典，我们忽略这个条目
                continue

    # 构建结果字典
    result = {
        'plugin': plugin_data.strip(),
        'answer': last_answer_content.strip()
    }
    
    return result

def extract_bracket_content(text):
    # 正则表达式模式，匹配{}及其中的内容，但不包括{}
    pattern = r'({.*?})'
    # 使用findall方法找到所有匹配项
    matches = re.findall(pattern, text)
    return matches

def plugin_text_process(ref_info):
    # 将Plugin输出转化为字典
    # key = ref_web_url，ref_web_name，ref_std，think_process，result
    text = ref_info.replace('\\n', '')
    text = text.replace('\\\\\\\\\\\\\\', '\\')
    text = text.replace('\\\\\\', '')
    text = text[2:-2]
    text = extract_bracket_content(text)[0]
    text=eval(text)
    return text

def get_bot_direct_reply(api_response):
    bot_answer = api_response.get('answer', "")
    bot_answer = json.loads(bot_answer)
    reflection = bot_answer.get('reflection', "")
    intermediate = bot_answer.get('intermediate', "")
    classificationId = bot_answer.get('classificationId', "")
    answer = bot_answer.get('answer', "")
    return answer, reflection, intermediate, classificationId

In [9]:
# 计算METEOR分数和BLEU分数
def calculate_metrics(reference, candidate):
    # 预先分词
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    # 计算METEOR分数
    score_m = meteor_score([reference_tokens], candidate_tokens)
    # 使用 SmoothingFunction 来避免得分为 0 的情况
    smooth = SmoothingFunction().method1
    # 计算句子级别的BLEU分数
    sentence_bleu_score = sentence_bleu(reference, candidate_tokens, smoothing_function=smooth)
    return score_m, sentence_bleu_score

# Sentence-BERT模型的指标计算
def calculate_metrics_sbert(reference, candidate, sbert_model):
    # 生成嵌入向量
    reference_embedding = sbert_model.encode(reference, convert_to_tensor=True)
    candidate_embedding = sbert_model.encode(candidate, convert_to_tensor=True)
    # 计算余弦相似度
    cosine_similarity = util.pytorch_cos_sim(reference_embedding, candidate_embedding)
    return cosine_similarity.item()

# 输出相似度得分

In [10]:
query = "一般目视检查后服务区左侧区域。"
expert = "Do a general visual inspection of left zone of the aft service area"
api_response = call_coze_api(query)
answer, reflection, intermediate, classificationId = get_bot_direct_reply(api_response)
print(f'问题：{query}')
# 输出
print(f'写作类别：{classificationId}')
print(f'中间结果：{intermediate}')
print(f'结果：{answer}')
print(f'修改意见：{reflection}')
# 计算METEOR分数和BLEU分数
s1_m_1, s1_b_1 = calculate_metrics(expert, answer)
s1_m_2, s1_b_2 = calculate_metrics(expert, intermediate)
print(f'S1-B-1: {s1_b_1}')
print(f'S1-M-1: {s1_m_1}')
print(f'S1-B-2: {s1_b_2}')
print(f'S1-M-2: {s1_m_2}')
s1_s_1 = calculate_metrics_sbert(expert, answer, sbert_model)
s1_s_2 = calculate_metrics_sbert(expert, intermediate, sbert_model)
print(f'S1-S-1: {s1_s_1}')
print(f'S1-S-2: {s1_s_2}')

问题：一般目视检查后服务区左侧区域。
写作类别：1
中间结果：After a general visual check, inspect the area. It is to the left of the service area.
结果：After a general visual check, inspect the area to the left of the service area.
修改意见：["Rule 3.6: Change 'Inspect the area to the left of the service area after a general visual check.' to 'After a general visual check, inspect the area to the left of the service area.'", 'Rule 5.1: Split the sentence to adhere to the 20-word limit.', 'Rule 5.4: Rearrange the sentence to place the condition first followed by a comma.', "Rule 2.3: Add 'the' before 'area' in 'inspect the area'.", "Rule 4.4: Combine the sentences using 'and' to improve readability and connection between the actions."]
S1-B-1: 0.011502783619900045
S1-M-1: 0.6140593329648055
S1-B-2: 0.009629943614188135
S1-M-2: 0.60061277822835
S1-S-1: 0.7907395958900452
S1-S-2: 0.7985110878944397


In [5]:
# 使用pandas读取CSV文件
df = pd.read_csv('./tests/trans_metrics_S1.csv', encoding='utf-8')
query_list = df['query'].tolist()
#for i in range(56,len(query_list)):
for i in range(len(query_list)):
    query = query_list[i]
    api_response = call_coze_api(query)
    answer, reflection, intermediate, classificationId = get_bot_direct_reply(api_response)
    if reflection != []:
        # 使用列表推导式将列表中的每个元素转换为字符串，并用换行符连接
        reflection = '<br>'.join(str(item) for item in reflection)
        reflection = "规则识别：<br>"+reflection
    else:
        reflection = ""
    print(f'问题：{query}')
    # 输出answer内容
    print(f'结果：{answer}')
    # 输出进度
    print(f'进度：{i+1}/{len(query_list)}')
    # 结果内容写入到df中
    df.loc[i, 'answer'] = answer
    df.loc[i, 'reflection'] = reflection
    df.loc[i, 'intermediate'] = intermediate
    df.loc[i, 'classificationId'] = classificationId
    # 计算METEOR分数和BLEU分数
    s1_m_1, s1_b_1 = calculate_metrics(df['expert'][i], answer)
    s1_m_2, s1_b_2 = calculate_metrics(df['expert'][i], intermediate)
    df.loc[i, 'S1-B-1'] = s1_b_1
    df.loc[i, 'S1-B-2'] = s1_b_2
    df.loc[i, 'S1-M-1'] = s1_m_1
    df.loc[i, 'S1-M-2'] = s1_m_2

# df结果另存为文件
df.to_csv('./tests/result_trans.csv', index=False, encoding='utf-8', mode='w')

问题：一般目视检查后服务区左侧区域。
结果：Generally inspect the area to the left of the service area by visual inspection.
进度：1/233
问题：收起1#、2#、3#多功能扰流板
结果：Fold in the spoilers 1, 2, and 3.
进度：2/233
问题：对左机翼外襟翼1#支臂整流罩区域内部做一般目视检
结果：Perform a general visual inspection inside the area of the left wing's outboard flap 1# support fairing.
进度：3/233
问题：可以从前起落架和主起落架牵引飞机。
结果：The plane can be towed using the nose landing gear or the main landing gear.
进度：4/233
问题：当牵引飞机时，必须保证牵引人员通信正常。防止设备突然移动导致人员伤亡。
结果：CAUTION: WHEN TOWING AN AIRCRAFT:
- ENSURE THE TOW PERSONNEL'S COMMUNICATION IS FUNCTIONING PROPERLY.
进度：5/233
问题：(16) 为飞机以下部位提供保护套并设置相应的警告旗，防止沙土、粉尘、碎片和火山灰等污染物进入：
结果：CAUTION: ENSURE PROTECTIVE COVERS ARE IN PLACE FOR THE FOLLOWING AIRCRAFT AREAS:
- [LIST AREA 1]
- [LIST AREA 2]
- [LIST AREA 3]

IF NOT DONE, DAMAGE FROM FOREIGN OBJECTS MAY OCCUR.
进度：6/233
问题：注: 如本地有效电台广播干扰短音监听， 此步骤测试可以选择一个无效电台频率。
结果：**Note:**  
- If local valid radio broadcasts interfere with the short tone monitoring, this test step can choose an invali